In [1]:
from scenic.simulators.gfootball import rl_interface
from stable_baselines3 import PPO
from scenic.simulators.gfootball.rl_interface import GFScenicEnv
import pretrain_template
#from gfootball_impala_cnn import GfootballImpalaCNN
import gym
from tqdm import tqdm
import numpy as np
import torch as th
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from stable_baselines3 import PPO, A2C, SAC, TD3
from stable_baselines3.common.evaluation import evaluate_policy

from torch.utils.data.dataset import Dataset, random_split
import os

from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from stable_baselines3.common.preprocessing import is_image_space
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from torch import nn
import torch as th
import torch
import os

/home/ubuntu/.local/lib/python3.8/site-packages/scenic/core/errors.py:157: UserWarning: unable to install sys.excepthook to format Scenic backtraces
  warnings.warn('unable to install sys.excepthook to format Scenic backtraces')


In [2]:
class GfootballImpalaCNN(BaseFeaturesExtractor):
    """
    gfootball_impala_cnn is architecture used in the paper
    (https://arxiv.org/pdf/1907.11180.pdf).
    It is illustrated in the appendix. It is similar to Large architecture
    from IMPALA paper; we use 4 big blocks instead of 3 though.
    """

    def __init__(self, observation_space: gym.spaces.Box, features_dim: int = 256):
        super(GfootballImpalaCNN, self).__init__(observation_space, features_dim)
        # We assume CxHxW images (channels first)
        # Re-ordering will be done by pre-preprocessing or wrapper
        assert is_image_space(observation_space), (
            "You should use CNN only with images"
        )
        assert features_dim==256, "To replicate the same network"
        n_input_channels = observation_space.shape[0]


        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        print("device: ", self.device)

        self.conv_layers_config = [(16, 2), (32, 2), (32, 2), (32, 2)]
        self.pool = nn.MaxPool2d(kernel_size=3, stride=2)


        self.conv_blocks = [
            nn.Conv2d(in_channels=n_input_channels, out_channels=16, kernel_size=3, stride=1, padding=1),
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=1, padding=1)
        ]
            

        

        #if "cuda" in self.device.type:
        #    self.conv_blocks = [c.cuda() for c in self.conv_blocks]

        #https://www.tensorflow.org/api_docs/python/tf/nn/pool  -> If padding = "SAME": output_spatial_shape[i] = ceil(input_spatial_shape[i] / strides[i])
        self.pools = [nn.MaxPool2d(kernel_size=3, stride=2, padding=1) for _ in range(4)]

        self.resblocks_1 = [
            self.create_basic_res_block(16, 16),
            self.create_basic_res_block(32, 32),
            self.create_basic_res_block(32, 32),
            self.create_basic_res_block(32, 32)
        ]
        self.resblocks_2 = [
            self.create_basic_res_block(16, 16),
            self.create_basic_res_block(32, 32),
            self.create_basic_res_block(32, 32),
            self.create_basic_res_block(32, 32)
        ]


        
        if "cuda" in self.device.type:
            self.conv_blocks = [c.cuda() for c in self.conv_blocks]
            self.resblocks_1 = [c.cuda() for c in self.resblocks_1]
            self.resblocks_2 = [c.cuda() for c in self.resblocks_2]

        self.relu = nn.ReLU()
        self.flatten = nn.Flatten()
        
        
        #hack so that all the weights are saved correctly, by default stable_baselines3 only saves weights all torch tensors
        #So it
        self.conv_0 = self.conv_blocks[0]
        self.conv_1 = self.conv_blocks[1]
        self.conv_2 = self.conv_blocks[2]
        self.conv_3 = self.conv_blocks[3]
        
        
        self.res_1_0 = self.resblocks_1[0]
        self.res_1_1 = self.resblocks_1[1]
        self.res_1_2 = self.resblocks_1[2]
        self.res_1_3 = self.resblocks_1[3]

        self.res_2_0 = self.resblocks_2[0]
        self.res_2_1 = self.resblocks_2[1]
        self.res_2_2 = self.resblocks_2[2]
        self.res_2_3 = self.resblocks_2[3]
        #################################################



        # Compute shape by doing one forward pass
        """
        with th.no_grad():
            n_flatten = self.feat_extract(
                th.as_tensor(observation_space.sample()[None]).float()
            )
            n_flatten = n_flatten.shape[1]
        """
        n_flatten = 960
        self.linear = nn.Sequential(nn.Linear(n_flatten, features_dim), nn.ReLU()) #n_flatten=960




    def create_basic_res_block(self, in_channel, out_channel):
        return nn.Sequential(
            nn.ReLU(),
            nn.Conv2d(in_channels=in_channel, out_channels=out_channel, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=out_channel, out_channels=out_channel, kernel_size=3, stride=1, padding=1),
        )

    def feat_extract(self, observations: th.Tensor) -> th.Tensor:
        #observations = observations.to(self.device)
        observations = observations.float()
        observations /= 255

        conv_out = observations
        for i in range(4):
            #print("", i)
            #print(" 1. conv_out.is_cuda() ", conv_out.is_cuda)
            #print("     conv block weight", self.conv_blocks[i].weight.is_cuda)
            conv_out = self.conv_blocks[i](conv_out)
            #print(" 2. conv_out.is_cuda() ", conv_out.is_cuda)
            conv_out = self.pools[i](conv_out)

            block_input = conv_out
            conv_out = self.resblocks_1[i](conv_out)
            conv_out += block_input

            block_input = conv_out
            conv_out = self.resblocks_2[i](conv_out)
            conv_out += block_input
            #print(" 3. conv_out.is_cuda() ", conv_out.is_cuda)

        #print(" before relu . conv_out.is_cuda() ", conv_out.is_cuda)
        conv_out = self.relu(conv_out)
        #print(" after relu . conv_out.is_cuda() ", conv_out.is_cuda)
        conv_out = self.flatten(conv_out)
        #print(" after flatten . conv_out.is_cuda() ", conv_out.is_cuda)
        return conv_out

    def forward(self, observations: th.Tensor) -> th.Tensor:
        conv_out = self.feat_extract(observations)
        conv_out = self.linear(conv_out)

        return conv_out
        



In [3]:

def get_weight_sum(model):
    """model is a PPO object"""
    
    tensors = [
        model.policy.features_extractor.conv_blocks[0].weight.sum().detach(),
        model.policy.features_extractor.conv_blocks[1].weight.sum().detach(),
        model.policy.features_extractor.conv_blocks[2].weight.sum().detach(),
        model.policy.features_extractor.conv_blocks[3].weight.sum().detach(),
        model.policy.features_extractor.resblocks_1[0][1].weight.sum().detach(),
        model.policy.features_extractor.resblocks_1[0][3].weight.sum().detach(),
        model.policy.features_extractor.resblocks_1[1][1].weight.sum().detach(),
        model.policy.features_extractor.resblocks_1[1][3].weight.sum().detach(),
        model.policy.features_extractor.resblocks_1[2][1].weight.sum().detach(),
        model.policy.features_extractor.resblocks_1[2][3].weight.sum().detach(),
        model.policy.features_extractor.resblocks_1[3][1].weight.sum().detach(),
        model.policy.features_extractor.resblocks_1[3][3].weight.sum().detach(),
        model.policy.features_extractor.resblocks_2[0][1].weight.sum().detach(),
        model.policy.features_extractor.resblocks_2[0][3].weight.sum().detach(),
        model.policy.features_extractor.resblocks_2[1][1].weight.sum().detach(),
        model.policy.features_extractor.resblocks_2[1][3].weight.sum().detach(),
        model.policy.features_extractor.resblocks_2[2][1].weight.sum().detach(),
        model.policy.features_extractor.resblocks_2[2][3].weight.sum().detach(),
        model.policy.features_extractor.resblocks_2[3][1].weight.sum().detach(),
        model.policy.features_extractor.resblocks_2[3][3].weight.sum().detach(),
    ]
    if tensors[0].is_cuda:
        saved_sums = [tens.cpu().numpy() for tens in tensors]
    else:
        saved_sums = [tens.numpy() for tens in tensors]

    return saved_sums

In [6]:
def pretrain_agent(
        student,
        env,
        expert_dataset,
        batch_size=64,
        epochs=10,
        scheduler_gamma=0.7,
        learning_rate=1.0,
        log_interval=100,
        no_cuda=True,
        seed=1,
        test_batch_size=64,
):
    train_size = int(0.8 * len(expert_dataset))

    test_size = len(expert_dataset) - train_size

    train_expert_dataset, test_expert_dataset = random_split(
        expert_dataset, [train_size, test_size]
    )

    print("test_expert_dataset: ", len(test_expert_dataset))
    print("train_expert_dataset: ", len(train_expert_dataset))


    use_cuda = th.cuda.is_available()
    th.manual_seed(seed)
    device = th.device("cuda" if use_cuda else "cpu")
    kwargs = {"num_workers": 1, "pin_memory": True} if use_cuda else {}

    if isinstance(env.action_space, gym.spaces.Box):
        criterion = nn.MSELoss()
    else:
        criterion = nn.CrossEntropyLoss()

    # Extract initial policy
    model = student.policy.to(device)

    def train(model, device, train_loader, optimizer):
        model.train()

        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()

            if isinstance(env.action_space, gym.spaces.Box):
                # A2C/PPO policy outputs actions, values, log_prob
                # SAC/TD3 policy outputs actions only
                if isinstance(student, (A2C, PPO)):
                    action, _, _ = model(data)
                else:
                    # SAC/TD3:
                    action = model(data)
                action_prediction = action.double()
            else:
                # Retrieve the logits for A2C/PPO when using discrete actions
                latent_pi, _, _ = model._get_latent(data)
                logits = model.action_net(latent_pi)
                action_prediction = logits
                target = target.long()

            loss = criterion(action_prediction, target)
            loss.backward()
            optimizer.step()
            if batch_idx % log_interval == 0:
                print(
                    "Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                        epoch,
                        batch_idx * len(data),
                        len(train_loader.dataset),
                        100.0 * batch_idx / len(train_loader),
                        loss.item(),
                    )
                )

    def test(model, device, test_loader):
        model.eval()
        test_loss = 0
        with th.no_grad():
            for data, target in test_loader:
                data, target = data.to(device), target.to(device)

                if isinstance(env.action_space, gym.spaces.Box):
                    # A2C/PPO policy outputs actions, values, log_prob
                    # SAC/TD3 policy outputs actions only
                    if isinstance(student, (A2C, PPO)):
                        action, _, _ = model(data)
                    else:
                        # SAC/TD3:
                        action = model(data)
                    action_prediction = action.double()
                else:
                    # Retrieve the logits for A2C/PPO when using discrete actions
                    latent_pi, _, _ = model._get_latent(data)
                    logits = model.action_net(latent_pi)
                    action_prediction = logits
                    target = target.long()

                test_loss = criterion(action_prediction, target)
        test_loss /= len(test_loader.dataset)
        print(f"Test set: Average loss: {test_loss:.4f}")

    # Here, we use PyTorch `DataLoader` to our load previously created `ExpertDataset` for training
    # and testing
    train_loader = th.utils.data.DataLoader(
        dataset=train_expert_dataset, batch_size=batch_size, shuffle=True, **kwargs
    )
    test_loader = th.utils.data.DataLoader(
        dataset=test_expert_dataset, batch_size=test_batch_size, shuffle=True, **kwargs,
    )

    # Define an Optimizer and a learning rate schedule.
    optimizer = optim.Adadelta(model.parameters(), lr=learning_rate)
    scheduler = StepLR(optimizer, step_size=1, gamma=scheduler_gamma)

    # Now we are finally ready to train the policy model.
    for epoch in range(1, epochs + 1):
        train(model, device, train_loader, optimizer)
        test(model, device, test_loader)
        scheduler.step()

    # Implant the trained policy network back into the RL student agent
    student.policy = model


In [7]:
def generate_expert_data(env, num_interactions=1000, save_file_name="expert_data"):

    expert_observations = []
    expert_actions = []

    obs = env.reset()

    for i in tqdm(range(num_interactions)):
        expert_observations.append(obs)

        obs, reward, done, info = env.step(env.action_space.sample())
        #print(info)
        action = info["action_taken"]
        expert_actions.append(action)

        if done:
            obs = env.reset()
    
    expert_observations = np.array(expert_observations)
    expert_observations = np.moveaxis(expert_observations, [3], [1])
    expert_actions = np.array(expert_actions)
    print("Expert observation shape: ", expert_observations.shape)
    print("Expert actions shape: ", expert_actions.shape)

    np.savez_compressed(
        save_file_name,
        expert_actions=expert_actions,
        expert_observations=expert_observations,
    )
    return expert_observations, expert_actions

In [10]:
cwd = os.getcwd()
print("Current Directory:", cwd)
rewards = "scoring"
target_scenario_name = f"{cwd}/pretrain/run_to_score.scenic"

save_dir = f"{cwd}/pretrain/saved_models_hp"
logdir = f"{cwd}/tboard/dev/pretrain"
tracedir = f"{cwd}/game_trace"

Current Directory: /home/ubuntu/ScenicGFootBall/rl_training


In [11]:
#create target environment
gf_env_settings = {
        "stacked": True,
        "rewards": rewards,
        "representation": 'extracted',
        "players": [f"agent:left_players=1"],
        "real_time": False,
        "action_set": "default"
    }

from scenic.simulators.gfootball.utilities.scenic_helper import buildScenario
scenario = buildScenario(target_scenario_name)
target_env = GFScenicEnv(initial_scenario=scenario, gf_env_settings=gf_env_settings)

pygame 2.0.1 (SDL 2.0.14, Python 3.8.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [27]:
#generate expert data
num_interactions=50000
expert_data_file = f"pretrain/expert_data/rts_{num_interactions}"


#create data_gen_environment
gf_env_settings = {
    "stacked": True,
    "rewards": 'scoring',
    "representation": 'extracted',
    "players": [f"agent:left_players=1"],
    "real_time": False,
    "action_set": "default",#"default" "v2"
}

datagen_scenario_file = f"{cwd}/pretrain/run_to_score_with_behave.scenic"
datagen_scenario = buildScenario(datagen_scenario_file)
from scenic.simulators.gfootball.rl_interface import GFScenicEnv

#import scenic.syntax.veneer as veneer
#veneer.reset()

datagen_env = GFScenicEnv(initial_scenario=datagen_scenario, gf_env_settings=gf_env_settings, use_scenic_behavior_in_step=True)
print("Mean Reward and STD of Scenic Behavior Agent", mean_perf_random_agent(datagen_env, num_trials=5))

expert_observations, expert_actions = generate_expert_data(datagen_env, num_interactions=num_interactions, save_file_name=expert_data_file) 



  0%|          | 0/5 [00:00<?, ?it/s]

Environment will ignore actions passed to step() and take action provided by Scenic


  0%|          | 19/50000 [00:00<04:31, 184.22it/s]

Mean Reward and STD of Scenic Behavior Agent (1.0, 0.0)


100%|██████████| 50000/50000 [05:32<00:00, 150.39it/s]


Expert observation shape:  (50000, 16, 72, 96)
Expert actions shape:  (50000,)


Pretrain Agent

In [13]:
#Create the PPO object with required parameters

ppo_agent, parameter_dict = pretrain_template.get_model_and_params(
    env=target_env, ALGO=PPO, features_extractor_class = GfootballImpalaCNN, scenario_name=target_scenario_name,
    logdir=logdir, override_params={}, rewards=rewards)

print("env (from model) observation space: ", ppo_agent.get_env().observation_space)


Using scoring Parameters
Using cuda device
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
device:  cuda:0
env (from model) observation space:  Box(0, 255, (16, 72, 96), uint8)


In [32]:
#Load expert Data
saved_exp_data = f"pretrain/expert_data/rts_{num_interactions}"
loaded_data = np.load(f"{saved_exp_data}.npz")
expert_observations = loaded_data["expert_observations"]
expert_actions = loaded_data["expert_actions"]
expert_dataset = ExpertDataSet(expert_observations, expert_actions)

print(f"Loaded data obs: {expert_observations.shape}, actions: {expert_actions.shape}")

Loaded data obs: (50000, 16, 72, 96), actions: (50000,)


In [ ]:
pretrain_agent(
    student=ppo_agent,
    env=target_env,
    expert_dataset=expert_dataset,
    epochs=5
)


In [ ]:
checsum_pretrained = np.sum(get_weight_sum(ppo_agent))
print(f"Checksum of pretrained agent: {checsum_pretrained}")

In [ ]:
mean_performance_pretrained = mean_perf_agent(agent=ppo_agent, env=target_env, num_trials=20)
print(mean_performance_pretrained)


In [ ]:
#save pretrained agent
saved_pretrained_agent_file = f"pretrain/saved_agents/pretrained_PPO_rts_{num_interactions}"
ppo_agent.save(saved_pretrained_agent_file)
del ppo_agent

<h1> HyperParameter Tuning </h1>

In [28]:
#Load expert Data
saved_exp_data = f"pretrain/expert_data/rts_{num_interactions}"
loaded_data = np.load(f"{saved_exp_data}.npz")
expert_observations = loaded_data["expert_observations"]
expert_actions = loaded_data["expert_actions"]
expert_dataset = ExpertDataSet(expert_observations, expert_actions)

print(f"Loaded data obs: {expert_observations.shape}, actions: {expert_actions.shape}")

Loaded data obs: (50000, 16, 72, 96), actions: (50000,)


In [29]:
from sklearn.model_selection import ParameterGrid

num_trials_evaluate = 20
param_grid = {'batch_size': [64, 128], 'learning_rate': [0.001, 0.0001, 0.0003, 0.00011879, 0.0002], "epochs": [20]}
hyperparameters = list(ParameterGrid(param_grid))


In [30]:
hyperparameters

[{'batch_size': 64, 'epochs': 20, 'learning_rate': 0.001},
 {'batch_size': 64, 'epochs': 20, 'learning_rate': 0.0001},
 {'batch_size': 64, 'epochs': 20, 'learning_rate': 0.0003},
 {'batch_size': 64, 'epochs': 20, 'learning_rate': 0.00011879},
 {'batch_size': 64, 'epochs': 20, 'learning_rate': 0.0002},
 {'batch_size': 128, 'epochs': 20, 'learning_rate': 0.001},
 {'batch_size': 128, 'epochs': 20, 'learning_rate': 0.0001},
 {'batch_size': 128, 'epochs': 20, 'learning_rate': 0.0003},
 {'batch_size': 128, 'epochs': 20, 'learning_rate': 0.00011879},
 {'batch_size': 128, 'epochs': 20, 'learning_rate': 0.0002}]

In [31]:

hp_results = [] 

for i, param in enumerate(hyperparameters):
    
    
    ppo_agent, parameter_dict = pretrain_template.get_model_and_params(
    env=target_env, ALGO=PPO, features_extractor_class = GfootballImpalaCNN, scenario_name=target_scenario_name,
    logdir=logdir, override_params={}, rewards=rewards)
    
    print(param)
    
    pretrain_agent(
        student=ppo_agent,
        env=target_env,
        expert_dataset=expert_dataset,
        **param)
    
    checsum_pretrained = np.sum(get_weight_sum(ppo_agent))
    print(f"Checksum of pretrained agent: {checsum_pretrained}")
    
    mean_performance_pretrained = mean_perf_agent(agent=ppo_agent, env=target_env, num_trials=num_trials_evaluate)
    print("BC Performance: ", mean_performance_pretrained)
    print("-"*80)
    print("")
    hp_results.append(mean_performance_pretrained)
    
    #save pretrained agent
    saved_pretrained_agent_file = f"pretrain/hp_saved_agents/pretrained_PPO_rts_{i}"
    ppo_agent.save(saved_pretrained_agent_file)
    del ppo_agent

Using scoring Parameters
Using cuda device
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
device:  cuda:0
{'batch_size': 64, 'epochs': 20, 'learning_rate': 0.001}
test_expert_dataset:  10000
train_expert_dataset:  40000
Train Epoch: 1 [0/40000 (0%)]	Loss: 2.944686
Train Epoch: 1 [6400/40000 (16%)]	Loss: 1.481545
Train Epoch: 1 [12800/40000 (32%)]	Loss: 1.251138
Train Epoch: 1 [19200/40000 (48%)]	Loss: 0.947303
Train Epoch: 1 [25600/40000 (64%)]	Loss: 1.275499
Train Epoch: 1 [32000/40000 (80%)]	Loss: 1.267398
Train Epoch: 1 [38400/40000 (96%)]	Loss: 1.106120
Test set: Average loss: 0.0001
Train Epoch: 2 [0/40000 (0%)]	Loss: 1.073400
Train Epoch: 2 [6400/40000 (16%)]	Loss: 1.276442
Train Epoch: 2 [12800/40000 (32%)]	Loss: 1.134003
Train Epoch: 2 [19200/40000 (48%)]	Loss: 1.055667
Train Epoch: 2 [25600/40000 (64%)]	Loss: 1.285265
Train Epoch: 2 [32000/40000 (80%)]	Loss: 1.201044
Train Epoch: 2 [38400/40000 (96%)]	Loss: 1.229675
Test set: Average loss: 0.0001
T

  0%|          | 0/20 [00:00<?, ?it/s]

Test set: Average loss: 0.0001
Checksum of pretrained agent: -1.3908171653747559


100%|██████████| 20/20 [00:34<00:00,  1.71s/it]


BC Performance:  (0.0, 0.0)
--------------------------------------------------------------------------------

Using scoring Parameters
Using cuda device
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
device:  cuda:0
{'batch_size': 64, 'epochs': 20, 'learning_rate': 0.0001}
test_expert_dataset:  10000
train_expert_dataset:  40000
Train Epoch: 1 [0/40000 (0%)]	Loss: 2.944711
Train Epoch: 1 [6400/40000 (16%)]	Loss: 2.944196
Train Epoch: 1 [12800/40000 (32%)]	Loss: 2.943420
Train Epoch: 1 [19200/40000 (48%)]	Loss: 2.940778
Train Epoch: 1 [25600/40000 (64%)]	Loss: 2.932578
Train Epoch: 1 [32000/40000 (80%)]	Loss: 2.902669
Train Epoch: 1 [38400/40000 (96%)]	Loss: 2.808774
Test set: Average loss: 0.0003
Train Epoch: 2 [0/40000 (0%)]	Loss: 2.752116
Train Epoch: 2 [6400/40000 (16%)]	Loss: 2.460526
Train Epoch: 2 [12800/40000 (32%)]	Loss: 1.751721
Train Epoch: 2 [19200/40000 (48%)]	Loss: 1.385153
Train Epoch: 2 [25600/40000 (64%)]	Loss: 1.173814
Train Epoch: 2 [32000

...

BC Performance:  (0.0, 0.0)
--------------------------------------------------------------------------------

Using scoring Parameters
Using cuda device
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
device:  cuda:0
{'batch_size': 128, 'epochs': 20, 'learning_rate': 0.0002}
test_expert_dataset:  10000
train_expert_dataset:  40000
Train Epoch: 1 [0/40000 (0%)]	Loss: 2.944744
Train Epoch: 1 [12800/40000 (32%)]	Loss: 2.943451
Train Epoch: 1 [25600/40000 (64%)]	Loss: 2.935115
Train Epoch: 1 [38400/40000 (96%)]	Loss: 2.840859
Test set: Average loss: 0.0003
Train Epoch: 2 [0/40000 (0%)]	Loss: 2.797559
Train Epoch: 2 [12800/40000 (32%)]	Loss: 1.958616
Train Epoch: 2 [25600/40000 (64%)]	Loss: 1.182811
Train Epoch: 2 [38400/40000 (96%)]	Loss: 1.161187
Test set: Average loss: 0.0001
Train Epoch: 3 [0/40000 (0%)]	Loss: 1.268270
Train Epoch: 3 [12800/40000 (32%)]	Loss: 1.244456
Train Epoch: 3 [25600/40000 (64%)]	Loss: 1.017338
Train Epoch: 3 [38400/40000 (96%)]	Loss: 1

  0%|          | 0/20 [00:00<?, ?it/s]

Test set: Average loss: 0.0001
Checksum of pretrained agent: -2.5195140838623047


100%|██████████| 20/20 [00:28<00:00,  1.43s/it]

BC Performance:  (0.0, 0.0)
--------------------------------------------------------------------------------



In [33]:
from sklearn.model_selection import ParameterGrid

num_trials_evaluate = 20
param_grid = {'batch_size': [64, 128], 'learning_rate': [0.001, 0.0001, 0.0003, 0.00011879, 0.0002], "epochs": [50]}
hyperparameters = list(ParameterGrid(param_grid))



In [34]:
hyperparameters

[{'batch_size': 64, 'epochs': 50, 'learning_rate': 0.001},
 {'batch_size': 64, 'epochs': 50, 'learning_rate': 0.0001},
 {'batch_size': 64, 'epochs': 50, 'learning_rate': 0.0003},
 {'batch_size': 64, 'epochs': 50, 'learning_rate': 0.00011879},
 {'batch_size': 64, 'epochs': 50, 'learning_rate': 0.0002},
 {'batch_size': 128, 'epochs': 50, 'learning_rate': 0.001},
 {'batch_size': 128, 'epochs': 50, 'learning_rate': 0.0001},
 {'batch_size': 128, 'epochs': 50, 'learning_rate': 0.0003},
 {'batch_size': 128, 'epochs': 50, 'learning_rate': 0.00011879},
 {'batch_size': 128, 'epochs': 50, 'learning_rate': 0.0002}]

In [35]:

hp_results2 = [] 

for i, param in enumerate(hyperparameters):
    
    
    ppo_agent, parameter_dict = pretrain_template.get_model_and_params(
    env=target_env, ALGO=PPO, features_extractor_class = GfootballImpalaCNN, scenario_name=target_scenario_name,
    logdir=logdir, override_params={}, rewards=rewards)
    
    print(param)
    
    pretrain_agent(
        student=ppo_agent,
        env=target_env,
        expert_dataset=expert_dataset,
        **param)
    
    checsum_pretrained = np.sum(get_weight_sum(ppo_agent))
    print(f"Checksum of pretrained agent: {checsum_pretrained}")
    
    mean_performance_pretrained = mean_perf_agent(agent=ppo_agent, env=target_env, num_trials=num_trials_evaluate)
    print("BC Performance: ", mean_performance_pretrained)
    print("-"*80)
    print("")
    hp_results2.append(mean_performance_pretrained)
    
    #save pretrained agent
    saved_pretrained_agent_file = f"pretrain/hp_saved_agents/pretrained_PPO_rts_2_{i}"
    ppo_agent.save(saved_pretrained_agent_file)
    del ppo_agent

Using scoring Parameters
Using cuda device
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
device:  cuda:0
{'batch_size': 64, 'epochs': 50, 'learning_rate': 0.001}
test_expert_dataset:  10000
train_expert_dataset:  40000
Train Epoch: 1 [0/40000 (0%)]	Loss: 2.944711
Train Epoch: 1 [6400/40000 (16%)]	Loss: 2.515022
Train Epoch: 1 [12800/40000 (32%)]	Loss: 1.459178
Train Epoch: 1 [19200/40000 (48%)]	Loss: 1.130676
Train Epoch: 1 [25600/40000 (64%)]	Loss: 1.180690
Train Epoch: 1 [32000/40000 (80%)]	Loss: 1.061220
Train Epoch: 1 [38400/40000 (96%)]	Loss: 1.195698
Test set: Average loss: 0.0001
Train Epoch: 2 [0/40000 (0%)]	Loss: 1.104435
Train Epoch: 2 [6400/40000 (16%)]	Loss: 1.059005
Train Epoch: 2 [12800/40000 (32%)]	Loss: 1.058126
Train Epoch: 2 [19200/40000 (48%)]	Loss: 1.290734
Train Epoch: 2 [25600/40000 (64%)]	Loss: 1.132046
Train Epoch: 2 [32000/40000 (80%)]	Loss: 1.207089
Train Epoch: 2 [38400/40000 (96%)]	Loss: 1.258307
Test set: Average loss: 0.0001
T

Train Epoch: 22 [0/40000 (0%)]	Loss: 1.284851
Train Epoch: 22 [6400/40000 (16%)]	Loss: 0.934415
Train Epoch: 22 [12800/40000 (32%)]	Loss: 1.167377
Train Epoch: 22 [19200/40000 (48%)]	Loss: 1.116055
Train Epoch: 22 [25600/40000 (64%)]	Loss: 0.897126
Train Epoch: 22 [32000/40000 (80%)]	Loss: 1.377011
Train Epoch: 22 [38400/40000 (96%)]	Loss: 1.017045
Test set: Average loss: 0.0001
Train Epoch: 23 [0/40000 (0%)]	Loss: 0.985947
Train Epoch: 23 [6400/40000 (16%)]	Loss: 1.201733
Train Epoch: 23 [12800/40000 (32%)]	Loss: 1.117430
Train Epoch: 23 [19200/40000 (48%)]	Loss: 1.118998
Train Epoch: 23 [25600/40000 (64%)]	Loss: 1.227021
Train Epoch: 23 [32000/40000 (80%)]	Loss: 0.937705
Train Epoch: 23 [38400/40000 (96%)]	Loss: 1.059555
Test set: Average loss: 0.0001
Train Epoch: 24 [0/40000 (0%)]	Loss: 1.090354
Train Epoch: 24 [6400/40000 (16%)]	Loss: 1.156551
Train Epoch: 24 [12800/40000 (32%)]	Loss: 1.139349
Train Epoch: 24 [19200/40000 (48%)]	Loss: 1.115640
Train Epoch: 24 [25600/40000 (64%)]	Lo

Train Epoch: 43 [25600/40000 (64%)]	Loss: 1.280851
Train Epoch: 43 [32000/40000 (80%)]	Loss: 1.204087
Train Epoch: 43 [38400/40000 (96%)]	Loss: 1.150907
Test set: Average loss: 0.0001
Train Epoch: 44 [0/40000 (0%)]	Loss: 1.123913
Train Epoch: 44 [6400/40000 (16%)]	Loss: 0.977728
Train Epoch: 44 [12800/40000 (32%)]	Loss: 0.979753
Train Epoch: 44 [19200/40000 (48%)]	Loss: 1.026369
Train Epoch: 44 [25600/40000 (64%)]	Loss: 1.201815
Train Epoch: 44 [32000/40000 (80%)]	Loss: 1.114139
Train Epoch: 44 [38400/40000 (96%)]	Loss: 1.202283
Test set: Average loss: 0.0001
Train Epoch: 45 [0/40000 (0%)]	Loss: 0.980199
Train Epoch: 45 [6400/40000 (16%)]	Loss: 1.067474
Train Epoch: 45 [12800/40000 (32%)]	Loss: 1.261053
Train Epoch: 45 [19200/40000 (48%)]	Loss: 1.159162
Train Epoch: 45 [25600/40000 (64%)]	Loss: 1.056508
Train Epoch: 45 [32000/40000 (80%)]	Loss: 1.278370
Train Epoch: 45 [38400/40000 (96%)]	Loss: 1.200320
Test set: Average loss: 0.0001
Train Epoch: 46 [0/40000 (0%)]	Loss: 1.148265
Train 

  0%|          | 0/20 [00:00<?, ?it/s]

Test set: Average loss: 0.0001
Checksum of pretrained agent: 1.1574864387512207


...

Train Epoch: 35 [25600/40000 (64%)]	Loss: 1.218310
Train Epoch: 35 [38400/40000 (96%)]	Loss: 1.082062
Test set: Average loss: 0.0002
Train Epoch: 36 [0/40000 (0%)]	Loss: 1.187372
Train Epoch: 36 [12800/40000 (32%)]	Loss: 1.221522
Train Epoch: 36 [25600/40000 (64%)]	Loss: 1.283674
Train Epoch: 36 [38400/40000 (96%)]	Loss: 0.963820
Test set: Average loss: 0.0001
Train Epoch: 37 [0/40000 (0%)]	Loss: 1.141381
Train Epoch: 37 [12800/40000 (32%)]	Loss: 0.940113
Train Epoch: 37 [25600/40000 (64%)]	Loss: 1.294082
Train Epoch: 37 [38400/40000 (96%)]	Loss: 1.109307
Test set: Average loss: 0.0001
Train Epoch: 38 [0/40000 (0%)]	Loss: 1.198196
Train Epoch: 38 [12800/40000 (32%)]	Loss: 1.226599
Train Epoch: 38 [25600/40000 (64%)]	Loss: 1.260898
Train Epoch: 38 [38400/40000 (96%)]	Loss: 1.106881
Test set: Average loss: 0.0001
Train Epoch: 39 [0/40000 (0%)]	Loss: 1.005476
Train Epoch: 39 [12800/40000 (32%)]	Loss: 1.204417
Train Epoch: 39 [25600/40000 (64%)]	Loss: 1.269809
Train Epoch: 39 [38400/40000 

  0%|          | 0/20 [00:00<?, ?it/s]

Test set: Average loss: 0.0002
Checksum of pretrained agent: -3.347726821899414


100%|██████████| 20/20 [00:37<00:00,  1.86s/it]

BC Performance:  (0.0, 0.0)
--------------------------------------------------------------------------------



<h1>Run normal training</h1>

In [ ]:
assert False

In [ ]:
#Load PPO Agent
saved_pretrained_agent_file = f"pretrain/saved_agents/pretrained_PPO_rts_{20000}"
loaded_PPO = PPO.load(saved_pretrained_agent_file)
checksum_pretrain_loaded = np.sum(get_weight_sum(loaded_PPO))
print(f"Checksum of loaded pretrained agent: {checksum_pretrain_loaded}")

In [ ]:
mean_performance_pretrained = mean_perf_agent(agent=loaded_PPO, env=target_env, num_trials=20)
print("Pretrained Agent Performance: ", mean_performance_pretrained)

In [ ]:
final_agent, parameter_dict = pretrain_template.get_model_and_params(
    env=target_env, ALGO=PPO, features_extractor_class = GfootballImpalaCNN, scenario_name=target_scenario_name,
    logdir=logdir, override_params={}, rewards=rewards)

#print("weight sum with default loading: ", np.sum(get_weight_sum(final_agent)))
final_agent.policy = loaded_PPO.policy
print("weight sum after initializing weight from saved policy: ", np.sum(get_weight_sum(final_agent)))
print("env (from model) observation space: ", final_agent.get_env().observation_space)

In [ ]:
mean_performance_warm = mean_perf_agent(agent=final_agent, env=target_env, num_trials=20)
print("Mean Performance Loaded Pretrained Agent", mean_performance_warm)

In [ ]:
total_training_timesteps = 5000

In [ ]:
pretrain_template.train(model=final_agent, parameters=parameter_dict,
                            n_eval_episodes=10, total_training_timesteps=total_training_timesteps,
                            eval_freq=10000,
                            save_dir=save_dir, logdir=logdir, dump_info={"rewards": rewards})

In [ ]:
mean_performance_warm = mean_perf_agent(agent=final_agent, env=target_env, num_trials=20)
print(mean_performance_warm)

In [ ]:
print("weight sum after initializing weight from saved policy: ", np.sum(get_weight_sum(final_agent)))

In [ ]:
#save final agent
saved_final_agent_file = f"pretrain/saved_agents/final_PPO_rts_{total_training_timesteps}"
final_agent.save(saved_final_agent_file)
del final_agent

In [ ]:
#Load RL Model
loaded_final = PPO.load(f"pretrain/saved_agents/final_PPO_rts_{total_training_timesteps}")

In [ ]:
print("weight sum after initializing weight from saved policy: ", np.sum(get_weight_sum(loaded_final)))
mean_performance_warm = mean_perf_agent(agent=loaded_final, env=target_env, num_trials=20)
print(mean_performance_warm)